<p style="font-family: 'Roboto', sans-serif; font-weight: bold; letter-spacing: normal; color: #005792; font-size: 16px; text-align: left; padding: 5px; border-bottom: 2px solid #007ACC; background-color: #F0F8FF;">
  Package Imports
</p>

In [1]:
# General imports
import re
import numpy as np
import pandas as pd
from warnings import filterwarnings
from tqdm.notebook import tqdm
from gc import collect
from IPython.display import clear_output, display, HTML
from termcolor import colored

# Configure pandas display options
pd.options.display.max_rows = 50
pd.set_option('display.float_format', '{:,.5f}'.format)

# Data processing and statistical analysis
from scipy.stats import mode, iqr, anderson, shapiro, normaltest
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Visualization libraries and settings
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
%matplotlib inline

# Set warnings filter (to ignore warnings)
filterwarnings('ignore')


In [2]:
# Model imports:-

from sklearn import datasets
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.feature_selection import mutual_info_regression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import FunctionTransformer, PowerTransformer, PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, PowerTransformer, RobustScaler

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LogisticRegression

<div style="color: #333333; 
           display: block; 
           border-radius: 12px;
           background-color: #FAFBFD; 
           font-size: 14px; 
           font-family: 'Roboto', sans-serif; 
           letter-spacing: 0.5px;
           padding: 20px; 
           border: 1px solid #CCCCCC; 
           ">
    
The data is generated by a deep learning model trained on the Machine Failure Predictions dataset. The relationships between the variables are akin to those in the original dataset but with notable distinctions.

We're delving into two distinct datasets: the **Training dataset**, which houses both predictor variables and the target variable; and the **Testing dataset**, carrying only the predictor variables.

The target variable, or our focus for prediction, is **Machine Failure**—a binary indicator revealing whether a machine has failed. The predictor variables at our disposal include:

- **`Product ID`**: Acts as a specific identifier for products or machines, potentially revealing if certain machines are more prone to failure.
- **`Type`**: The category of machine, which could influence its likelihood of failure.
- **`Air temperature [K]`** and **`Process temperature [K]`**: Critical in forecasting machine failure, as overheating is a common failure cause.
- **`Rotational speed [rpm]`**: The operation speed, with higher speeds possibly leading to increased failure risks.
- **`Torque [Nm]`**: The operational force, where excessive torque can signify wear and tear, leading to failure.
- **`Tool wear [min]`**: The usage duration of tools, where longer use periods might forecast imminent failure.

Additionally, we assess specific types of failures, each marked by a binary indicator:
- **`TWF`** (Tool Wear Failure),
- **`HDF`** (Heat Dissipation Failure),
- **`PWF`** (Power Failure),
- **`OSF`** (Overstrain Failure),
- **`RNF`** (Random Failure).

Should any of these failure indicators be triggered, it signals the machine's functional demise.

**Evaluating Our Model's Performance**

We're evaluating the model's effectiveness primarily based on the area under the ROC curve, which compares predicted probabilities to actual outcomes.

**Analysis Phases**

**Phase 1: Data Examination**

Our initial analysis stage encompasses a thorough dataset review—identifying and rectifying missing values, evaluating feature distribution, spotting outliers, and conducting normality tests to ensure data quality and reliability.

**Phase 2: Data Correction**

This stage aims at addressing and amending early-detected anomalies, crucial for enhancing the dataset's overall quality and setting a solid foundation for predictive modeling.

**Phase 3: Predictive Modeling**

Concluding our analysis, we embark on developing and deploying a **binary classification model**, aspiring to accurately predict machine failures and measure our success against the precision of the area under the ROC curve.

</div>

<p style="font-family: 'Roboto', sans-serif; font-weight: bold; letter-spacing: normal; color: #005792; font-size: 16px; text-align: left; padding: 5px; border-bottom: 2px solid #007ACC; background-color: #F0F8FF;">
  Assigning global variables
</p>

In [3]:
# Assigning global variables to be used throughout:-
target = 'Machine failure'

grid_specs = {'visible': True, 'which': 'both', 'linestyle': '--',
              'color': 'lightgrey', 'linewidth': 0.50}
title_specs = {'fontsize': 14, 'fontweight': 'bold', 'color': 'tab:blue'}

def PrintColor(text:str, color:str = 'blue', attrs:list = ['bold', 'dark']):
    """
    Prints color outputs using termcolor-colored using a text F-string
    """
    print(colored(text, color = color, attrs = attrs))

<p style="font-family: 'Roboto', sans-serif; font-weight: bold; letter-spacing: normal; color: #005792; font-size: 16px; text-align: left; padding: 5px; border-bottom: 2px solid #007ACC; background-color: #F0F8FF;">
  Importing Train and Test Dataset
</p>

In [4]:
%%time

# Importing the data:-
xytrain = pd.read_csv('../data/train.csv', encoding = 'utf8')
xtest = pd.read_csv('../data/test.csv', encoding = 'utf8')

# Displaying the datasets:-
PrintColor(f"\nTrain data\n")
display(xytrain.head(5))

PrintColor(f"\nTest data\n")
display(xtest.head(5))

PrintColor(f"\nExtracting features from Test data\n")

features = xtest.columns
display(features)

print();


Train data



,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.60000,309.60000,1596,36.10000,140,0,0,0,0,0,0
1,1,M20343,M,302.60000,312.10000,1759,29.10000,200,0,0,0,0,0,0
2,2,L49454,L,299.30000,308.50000,1805,26.50000,25,0,0,0,0,0,0
3,3,L53355,L,301.00000,310.90000,1524,44.30000,197,0,0,0,0,0,0
4,4,M24050,M,298.00000,309.00000,1641,35.40000,34,0,0,0,0,0,0



Test data



,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,136429,L50896,L,302.30000,311.50000,1499,38.00000,60,0,0,0,0,0
1,136430,L53866,L,301.70000,311.00000,1713,28.80000,17,0,0,0,0,0
2,136431,L50498,L,301.30000,310.40000,1525,37.70000,96,0,0,0,0,0
3,136432,M21232,M,300.10000,309.60000,1479,47.60000,5,0,0,0,0,0
4,136433,M19751,M,303.40000,312.30000,1515,41.30000,114,0,0,0,0,0



Extracting features from Test data



Index(['id', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'],
      dtype='object')


CPU times: total: 78.1 ms
Wall time: 255 ms


<p style="font-family: Arial, sans-serif; font-weight: bold; color: #005792; font-size: 16px; text-align: left; border-bottom: 2px solid #007ACC; background-color: #F0F8FF;">  
Data Preprocessing
</p>


In [5]:
PrintColor(f"\nExtracting unique records per feature from Train and Test data\n")

PrintColor(f"\nTrain data unique records\n")
display(xytrain.nunique())

PrintColor(f"\nTest data unique records\n")
display(xtest.nunique())


Extracting unique records per feature from Train and Test data


Train data unique records



id                         136429
Product ID                   9976
Type                            3
Air temperature [K]            95
Process temperature [K]        81
Rotational speed [rpm]        952
Torque [Nm]                   611
Tool wear [min]               246
Machine failure                 2
TWF                             2
HDF                             2
PWF                             2
OSF                             2
RNF                             2
dtype: int64


Test data unique records



id                         90954
Product ID                  9909
Type                           3
Air temperature [K]           92
Process temperature [K]       84
Rotational speed [rpm]       946
Torque [Nm]                  595
Tool wear [min]              246
TWF                            2
HDF                            2
PWF                            2
OSF                            2
RNF                            2
dtype: int64

<p style="font-family: Arial, sans-serif; font-weight: bold; color: #005792; font-size: 16px; text-align: left; border-bottom: 2px solid #007ACC; background-color: #F0F8FF;">  Dropping the "id" column from Train and Test data

</p>

In [10]:
xytrain = xytrain.drop('id', axis=1)
xtest = xtest.drop('id', axis=1)

In [11]:
PrintColor(f'Train Data set contains {xytrain.duplicated().sum()} duplicated rows')

PrintColor(f'\nTest Data set contains {xtest.duplicated().sum()} duplicated rows\n')

Train Data set contains 1134 duplicated rows

Test Data set contains 523 duplicated rows



<p style="font-family: Arial, sans-serif; font-weight: bold; color: #005792; font-size: 16px; text-align: left; border-bottom: 2px solid #007ACC; background-color: #F0F8FF;">  Dropping duplicated rows from Train and Test data

</p>

In [13]:
xytrain = xytrain.drop_duplicates()

xtest = xtest.drop_duplicates()

In [12]:
PrintColor(f"\nData information - Train and Test data")

PrintColor(f"\nTrain data information\n")
display(xytrain.info())

PrintColor(f"\nTest data information\n")
display(xtest.info())


Data information - Train and Test data

Train data information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136429 entries, 0 to 136428
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Product ID               136429 non-null  object 
 1   Type                     136429 non-null  object 
 2   Air temperature [K]      136429 non-null  float64
 3   Process temperature [K]  136429 non-null  float64
 4   Rotational speed [rpm]   136429 non-null  int64  
 5   Torque [Nm]              136429 non-null  float64
 6   Tool wear [min]          136429 non-null  int64  
 7   Machine failure          136429 non-null  int64  
 8   TWF                      136429 non-null  int64  
 9   HDF                      136429 non-null  int64  
 10  PWF                      136429 non-null  int64  
 11  OSF                      136429 non-null  int64  
 12  RNF                      136429 non-null  int64  

None


Test data information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90954 entries, 0 to 90953
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product ID               90954 non-null  object 
 1   Type                     90954 non-null  object 
 2   Air temperature [K]      90954 non-null  float64
 3   Process temperature [K]  90954 non-null  float64
 4   Rotational speed [rpm]   90954 non-null  int64  
 5   Torque [Nm]              90954 non-null  float64
 6   Tool wear [min]          90954 non-null  int64  
 7   TWF                      90954 non-null  int64  
 8   HDF                      90954 non-null  int64  
 9   PWF                      90954 non-null  int64  
 10  OSF                      90954 non-null  int64  
 11  RNF                      90954 non-null  int64  
dtypes: float64(3), int64(7), object(2)
memory usage: 8.3+ MB


None

<div style="color: #333; border-radius: 12px; background-color: #fafbfd; font-size: 14px; font-family: 'Arial', sans-serif; padding: 20px; border: 1px solid #ccc;">
     The 'id' column, serving as a generic identifier, has been removed from both the Train and Test datasets. We identified and eliminated a total of 1,134 duplicate entries in the Train and 523 duplicates in the Test dataset. Additionally, both datasets are free from any null values.
</di>


<p style="font-family: Arial, sans-serif; font-weight: bold; color: #005792; font-size: 16px; text-align: left; border-bottom: 2px solid #007ACC; background-color: #F0F8FF;">  Creating a function to offer a comprehensive summary of the Training and Testing datasets.

</p>

In [32]:
def describe_dataframes(*dataframes, percentiles, target):
    """
    Generate and display descriptive statistics for given dataframes.

    Parameters:
    - dataframes: Tuples of DataFrame and its corresponding source name.
    - percentiles: List of percentiles to calculate.
    - target: The target variable to exclude from the description.
    """
    # Initialize an empty list to hold the modified dataframes
    described_dfs = [
        df.drop(target, axis=1, errors='ignore')
        .describe(percentiles=percentiles)
        .transpose()
        .drop('count', axis=1)
        .assign(source=source)
        .reset_index()
        .set_index(['index', 'source'])
        for df, source in dataframes
    ]
    # Concatenate and display the results
    all_described = pd.concat(described_dfs).sort_index(ascending=[True, False])
    display(all_described.style.format('{:.2f}'))

In [33]:
percentiles = [0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.95, 0.99]

PrintColor('\nData description - Train and Test data\n')

describe_dataframes((xytrain,'Train'),(xtest,'Test'), percentiles=percentiles, target=target)


Data description - Train and Test data

